<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/banner_IA.png"  width="1000px" height="200px">

# **Taller 05:  Classification**

## **Outline**

1. [Ejercicio 1.](#eje1)
2. [Ejercicio 2.](#eje2)
3. [Ejercicio 3.](#eje3)
4. [Ejercicio 4.](#eje4)
5. [Ejercicio 5.](#eje5)



In [2]:
#@title **Execute this cell**
#@markdown Please include your student id
import sys
import inspect

group_id = "IA1-20252-G1" #@param {type:"string"}
assignment_id = group_id +'.taller_classification'
student_id = "2204002" #@param {type:"string"}
"""
Put your student ID here

Example: student_id =  '2152145'
"""

"\nPut your student ID here\n\nExample: student_id =  '2152145'\n"

In [3]:
 #@title **Execute this cell**
#@markdown **UTILS**
#@markdown Please dont modify any line in this cell

import os
import json
import requests
from collections import namedtuple


Config = namedtuple('Config', ['server_name'])
config = Config(server_name='https://bivlabgrader.azurewebsites.net/api')


def check_solution_and_evaluate(assignment_id: str, student_func_str: str):

    # Set the endpoint and payload.
    payload = {
        'func_str': student_func_str,
        'assignment_id': assignment_id,
        'student_id': student_id
    }
    endpoint_url = config.server_name + '/CheckAndEvaluateSolution'
    # print(endpoint_url)

    # Make request to server with the data coming from the notebook.
    r = requests.post(endpoint_url, params=payload)
    pprint_json_response(r.json())
    return r


def pprint_json_response(response, indent=0):
    """Pretty print the response."""
    for key, value in response.items():
        print('\t' * indent + str(key.capitalize()))

        # If dictionary, do a recurrent call.
        if isinstance(value, dict):
            pprint_json_response(value, indent + 1)
        else:
            # Enumerate elements if list.
            if isinstance(value, list):
                if len(value) == 1:
                    print('\t' * (indent + 1) + str(value[0]))
                else:
                    for i, e in enumerate(value, start=1):
                        print('\t' * (indent + 1) + f'{i}. {e}')
            else:
                print('\t' * (indent + 1) + str(value))

In [4]:
#@title **Import libraries**

import matplotlib.pyplot as plt
import numpy as np
np.random.seed(21)

import warnings
warnings.filterwarnings('ignore')

---
# **Ejercicio 1**  <a name="eje1"></a>
---

# Clasificación

## Contexto
El dataset "Dogs of Zurich" (https://www.kaggle.com/datasets/kmader/dogs-of-zurich) contiene información sobre datos de dueños y mascotas en la ciudad de Zurich.

In [5]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1z0DCJFIFyLIObEBjhsm66RoVwRq7YYoj", sep=";")
df.head()

,HALTER_ID,ALTER,GESCHLECHT,STADTKREIS,STADTQUARTIER,RASSE1,RASSENTYP,GEBURTSJAHR_HUND,GESCHLECHT_HUND,HUNDEFARBE
0,126,51-60,m,9.0,92.0,Welsh Terrier,K,2011,w,schwarz/braun
1,171,61-70,m,3.0,31.0,Berner Sennenhund,I,2009,m,tricolor
2,574,61-70,w,2.0,23.0,Cairn Terrier,K,2002,w,brindle
3,695,41-50,m,6.0,63.0,Labrador Retriever,I,2012,w,braun
4,893,61-70,w,7.0,71.0,Mittelschnauzer,I,2010,w,schwarz


## Tu tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Considere como ground truth a la columna ["GESCHLECHT"] (el género del dueño de la mascota).
- Elimine a las columnas `["HALTER_ID", "ALTER", "STADTKREIS", "STADTQUARTIER"]`
- Reemplace los valores "m" por cero (0), y "w" por uno (1).
- Convierta las columnas categóricas `["RASSE1"], ["RASSENTYP"], ["HUNDEFARBE"]` a numéricas.
- Destine un 80% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene un estimador `RandomForestClassifier` con parámetros `n_estimators=100` y `random_state=21`
- **Devuelva** el `accuracy_score`

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [10]:
#@title **code student**

def taller05_20252_p01(df):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score

    # Eliminar valores nulos
    df.dropna(inplace=True)

    # Ground true
    target = "GESCHLECHT"

    # Elimuinar columnas
    df.drop(columns=["HALTER_ID", "ALTER", "STADTKREIS", "STADTQUARTIER"], inplace=True)

    # Valores m -> 0 w -> 1
    mapeo = {"m": 0, "w": 1}
    df["GESCHLECHT"] = df["GESCHLECHT"].replace(mapeo)
    df["GESCHLECHT_HUND"] = df["GESCHLECHT_HUND"].replace(mapeo)

    # Conversión a categoricas

    df["RASSE1"] = pd.factorize(df["RASSE1"])[0]
    df["RASSENTYP"] = pd.factorize(df["RASSENTYP"])[0]
    df["HUNDEFARBE"] = pd.factorize(df["HUNDEFARBE"])[0]

    X = df.drop(columns=[target])
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

    rf = RandomForestClassifier(n_estimators=100, random_state=21)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)

    return acc

In [11]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1z0DCJFIFyLIObEBjhsm66RoVwRq7YYoj", sep=";")
taller05_20252_p01(df)

0.6096681096681097

## ✅ Salidas esperadas

0.6096681096681097

In [12]:
#@title **send your answer**
student_func_str = inspect.getsource(taller05_20252_p01)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, you got the highest score.
Status
	You have achieved your best score: 5.0


---
# **Ejercicio 2**  <a name="eje2"></a>
---

# Clasificación

## Contexto
El dataset "80 Cereals" (https://www.kaggle.com/datasets/crawford/80-cereals) contiene información sobre distintas marcas de cereales, sus fabricantes, información nutricional, entre otros.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


## Tu Tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:
- Elimine los valores nulos del dataset.
- Considere como ground truth a la columna `["type"]` (el tipo de cereal).
- Elimine la columna `["name"]`
- Convierta la columna categórica `["mfr"]` a numérica.
- Destine un 85% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene un estimador `SVC` con parámetros `kernel="linear"` y `random_state=21`
- **Devuelva** el `accuracy_score` para este estimador.
- Entrene un estimador `RandomForestClassifier` con parámetros `n_estimators=100` y `random_state=21`
- **Devuelva** el `accuracy_score` para este estimador.


<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [ ]:
def taller05_20252_p02(df):
    import pandas as pd
    import numpy as np
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    np.random.seed(21)

    # Eliminar valores nulos
    df = df.dropna()

    # Ground truth (type)
    y = df["type"]

    # Eliminar la columna name y la columna de respuesta
    X = df.drop(columns=["name", "type"])

    # Convertir "mfr" a numérica
    X["mfr"] = pd.factorize(X["mfr"])[0]

    # Separar en entrenamiento (85%) y test (15%)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.15, random_state=21
    )

    # Entrenar SVC
    svc = SVC(kernel="linear", random_state=21)
    svc.fit(X_train, y_train)
    svc_acc = accuracy_score(y_test, svc.predict(X_test))

    # Entrenar RandomForest
    rf = RandomForestClassifier(n_estimators=100, random_state=21)
    rf.fit(X_train, y_train)
    rf_acc = accuracy_score(y_test, rf.predict(X_test))

    return svc_acc, rf_acc

In [ ]:
#@title **check your answer**
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1DtVWAAUMDzVkh0NqDQT0i1JvTD8B4txv")
taller05_20252_p02(df)

(0.8333333333333334, 0.9166666666666666)

## ✅ Salidas esperadas

(0.8333333333333334, 0.9166666666666666)

In [ ]:
#@title **send your answer**
student_func_str = inspect.getsource(taller05_20252_p02)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both accuracy values are correct.


---
# **Ejercicio 3**  <a name="eje3"></a>
---

## Feature importances

## Contexto
El dataset `Animal Bites` (https://www.kaggle.com/datasets/rtatman/animal-bites) contiene información asociada a mordeduras de diferentes animales, datos de las víctimas y otros aspectos relevantes.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1-u5iid63dsDgwTR9rAeeHB4198K1I1Yt")
df.head()

,bite_date,SpeciesIDDesc,BreedIDDesc,GenderIDDesc,color,vaccination_yrs,vaccination_date,victim_zip,AdvIssuedYNDesc,WhereBittenIDDesc,quarantine_date,DispositionIDDesc,head_sent_date,release_date,ResultsIDDesc
0,1985-05-05 00:00:00,DOG,NaN,FEMALE,LIG. BROWN,1.0,1985-06-20 00:00:00,40229,NO,BODY,1985-05-05 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
1,1986-02-12 00:00:00,DOG,NaN,UNKNOWN,BRO & BLA,NaN,NaN,40218,NO,BODY,1986-02-12 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
2,1987-05-07 00:00:00,DOG,NaN,UNKNOWN,NaN,NaN,NaN,40219,NO,BODY,1990-05-07 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
3,1988-10-02 00:00:00,DOG,NaN,MALE,BLA & BRO,NaN,NaN,NaN,NO,BODY,1990-10-02 00:00:00,UNKNOWN,NaN,NaN,UNKNOWN
4,1989-08-29 00:00:00,DOG,NaN,FEMALE,BLK-WHT,NaN,NaN,NaN,NO,BODY,NaN,UNKNOWN,NaN,NaN,UNKNOWN


## Tu tarea
Desarrolle una función que **reciba** un dataset (`df`) y que:

- Elimine las columnas `["BreedIDDesc", "vaccination_yrs", "vaccination_date", "bite_date", "release_date", "ResultsIDDesc", "head_sent_date", "DispositionIDDesc", "quarantine_date"]`
- Elimine los valores nulos del dataset.
- Convierta las columnas categóricas `["SpeciesIDDesc"], ["GenderIDDesc"],["color"] ["AdvIssuedYNDesc"], ["WhereBittenIDDesc"], ["victim_zip"]` a numéricas.
- Considere como ground truth a la columna `["GenderIDDesc"]` (el género del animal).
- Destine un 90% del dataset para el proceso de entrenamiento, usando también el parámetro `random_state=21`
- Entrene un estimador `DecisionTreeClassifier` con parámetros `random_state=21`
- **Devuelva** el `accuracy_score` para este estimador.
- Calcule las columnas más importantes del dataset.

Su nuevo dataset será: las 2 (dos) columnas <ins>**MÁS IMPORTANTES**</ins>.<br>Con esta nueva configuración:
- Entrene un estimador `RandomForestClassifier` con parámetros `n_estimators=100` y `random_state=21`
- **Devuelva** el `accuracy_score` para este estimador.

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [ ]:
def taller05_20252_p03(df):
    import numpy as np
    import pandas as pd
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score

    np.random.seed(21)

    # Elimina columnas especificadas
    drop_cols = [
        "BreedIDDesc", "vaccination_yrs", "vaccination_date", "bite_date", "release_date",
        "ResultsIDDesc", "head_sent_date", "DispositionIDDesc", "quarantine_date"
    ]
    df = df.drop(columns=drop_cols)

    # Elimina filas con valores nulos
    df = df.dropna()

    # Factoriza columnas categóricas
    cat_cols = [
        "SpeciesIDDesc", "GenderIDDesc", "color", "AdvIssuedYNDesc",
        "WhereBittenIDDesc", "victim_zip"
    ]
    for col in cat_cols:
        df[col] = pd.factorize(df[col])[0]

    # Columna objetivo
    y = df["GenderIDDesc"]
    X = df.drop(columns=["GenderIDDesc"])

    # Split 90/10
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=21
    )

    # DecisionTreeClassifier
    dt = DecisionTreeClassifier(random_state=21)
    dt.fit(X_train, y_train)
    dt_pred = dt.predict(X_test)
    dt_acc = accuracy_score(y_test, dt_pred)

    # Importancias y selección de las 2 más importantes
    importancias = dt.feature_importances_
    idx = np.argsort(importancias)[-2:]
    top_cols = X.columns[idx].tolist()

    # Solo con las 2 columnas más importantes
    X2 = df[top_cols]
    y2 = df["GenderIDDesc"]
    X2_train, X2_test, y2_train, y2_test = train_test_split(
        X2, y2, test_size=0.1, random_state=21
    )

    # RandomForestClassifier
    rf = RandomForestClassifier(n_estimators=100, random_state=21)
    rf.fit(X2_train, y2_train)
    rf_pred = rf.predict(X2_test)
    rf_acc = accuracy_score(y2_test, rf_pred)

    return dt_acc, rf_acc

In [ ]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1-u5iid63dsDgwTR9rAeeHB4198K1I1Yt")
taller05_20252_p03(df)

(0.4, 0.45294117647058824)

## ✅ Salidas esperadas

(0.4, 0.45294117647058824)

In [ ]:
#@title **send your answer**
student_func_str = inspect.getsource(taller05_20252_p03)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, both accuracy values are correct.


---
# **Ejercicio 4**  <a name="eje4"></a>
---

## Cross Validation

## Contexto
El dataset `supermarket sales` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/supermarket_sales.csv) contiene información relacionada con productos, costos, ganancias y otros aspectos de un supermercado.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## Tu tarea

Implementar una función que **reciba** como parámetro el dataset (`df`), y que:
- Elimine las columnas `["Invoice ID", "Date", "Time"]`
- Convierta las columnas categóricas `["Branch"], ["City"], ["Customer type"], ["Gender"], ["Product line"], ["Payment"]` a numéricas.
- El ground truth será la columna `["Customer type"]`
- Utilice un 80% del dataset para entrenamiento, con parámetros `test_size=0.2, random_state=21`)
- Entrene un estimador SVC con parámetros `kernel="poly", random_state=21`
- **Devuelva** el `accuracy_score` del estimador
- Entrene el mismo estimador, mediante `cross validation` con parámetros `cv=KFold(2, shuffle=False), scoring=make_scorer(accuracy_score)`
- **Devuelva** el promedio de los resultados obtenidos con este método
- **Devuelva** la desviación de los resultados obtenidos con este método

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [ ]:
def taller05_20252_p04(df):
    import numpy as np
    import pandas as pd
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split, cross_val_score, KFold
    from sklearn.metrics import accuracy_score, make_scorer

    np.random.seed(21)

    # 1. Elimina columnas innecesarias
    df = df.drop(columns=["Invoice ID", "Date", "Time"])

    # 2. Convierte las variables categóricas a numéricas
    cat_cols = ["Branch", "City", "Customer type", "Gender", "Product line", "Payment"]
    for col in cat_cols:
        df[col] = pd.factorize(df[col])[0]

    # 3. Separa variables predictoras y ground truth
    X = df.drop(columns=["Customer type"])
    y = df["Customer type"]

    # 4. División entrenamiento/prueba (80/20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=21
    )

    # 5. SVC y accuracy sobre set de prueba
    svc = SVC(kernel="poly", random_state=21)
    svc.fit(X_train, y_train)
    accuracy_simple = accuracy_score(y_test, svc.predict(X_test))

    # 6. Cross-validation con KFold(2)
    kf = KFold(n_splits=2, shuffle=False)
    cv_results = cross_val_score(
        svc, X, y, cv=kf, scoring=make_scorer(accuracy_score)
    )
    promedio = np.mean(cv_results)
    desviacion = np.std(cv_results)

    return accuracy_simple, promedio, desviacion

In [ ]:
#@title **check your answer**
import numpy as np
import pandas as pd

df = pd.read_csv("https://drive.google.com/uc?id=1Xd0hiXjeW2C3h2-zwve5ojrDGSU_F-8I")
taller05_20252_p04(df)

(0.485, np.float64(0.483), np.float64(0.0010000000000000009))

## ✅ Salidas esperadas

(0.485, np.float64(0.483), np.float64(0.0010000000000000009))

In [ ]:
#@title **send your answer**
student_func_str = inspect.getsource(taller05_20252_p04)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all accuracy values are correct.


---
# **Ejercicio 5**  <a name="eje5"></a>
---

# Cross validation: leave one out

## Contexto:
Considere el dataset `penguins size` (https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/penguins_size.csv) el cual contiene datos sobre las dimensiones de una población de pinguinos.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


## Tu tarea
Implemente una función que **reciba** un dataset (`df`) y que:
- **Elimine** los valores nulos del dataset.
- Convierta las columnas categóricas `["species"], ["island"], ["sex"]` a numéricas.
- Considere a la columna `["sex"]` como el ground truth.
- Lleve a cabo un entrenamiento mediante la variante `leave one out` de cross validation.
- El estimador a ser entrenado debe ser un `SVC` con parámetros `kernel="poly", random_state=21`
- Determine la cantidad de iteraciones necesaria para ser considerado `leave one out`.<br>
**Devuelva** dicha cantidad.
- La métrica a evaluar será `accuracy_score`
- **Devuelva** el promedio de los resultados obtenidos
- **Devuelva** la desviación de los resultados obtenidos

<br>

<ins>**Nota:**</ins> puede utilizar la función `pd.factorize` de pandas para convertir columnas categóricas, a numéricas:

https://pandas.pydata.org/docs/reference/api/pandas.factorize.html

In [ ]:
def taller05_20252_p05(df):
    import pandas as pd
    import numpy as np
    from sklearn.svm import SVC
    from sklearn.model_selection import LeaveOneOut, cross_val_score
    from sklearn.metrics import accuracy_score, make_scorer

    np.random.seed(21)

    # Elimina valores nulos
    df = df.dropna()

    # Convierte columnas categóricas a numéricas
    cat_cols = ["species", "island", "sex"]
    for col in cat_cols:
        df[col] = pd.factorize(df[col])[0]

    # X e y
    X = df.drop(columns=["sex"])
    y = df["sex"]

    # Leave One Out Cross Validation
    loo = LeaveOneOut()
    scores = cross_val_score(
        SVC(kernel="poly", random_state=21),
        X, y, cv=loo, scoring=make_scorer(accuracy_score)
    )

    n_iter = len(scores)
    promedio = np.mean(scores)
    desviacion = np.std(scores)

    return n_iter, promedio, desviacion

In [ ]:
#@title **check your answer**
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("https://drive.google.com/uc?id=1y84HuLlwI8mmibaM1KT2Q1puWwn32Xq1")
taller05_20252_p05(df)

(334, np.float64(0.5778443113772455), np.float64(0.4939030908854514))

## ✅ Salidas esperadas

(334, np.float64(0.5778443113772455), np.float64(0.4939030908854514))


In [ ]:
#@title **send your answer**
student_func_str = inspect.getsource(taller05_20252_p05)
r = check_solution_and_evaluate(assignment_id, student_func_str)

Score
	5.0
Message
	Excellent, all leave-one-out metrics are correct.


---
<img src="https://gitlab.com/bivl2ab/academico/cursos-uis/ai/ai-uis-student/raw/master/imgs/bannerThanks.jpg" alt="Drawing" style="width:700px;"/>